# Load Zarr Image from a public S3 store and analyze it in parallel

The images are taken from  the paper "In Toto Imaging and Reconstruction of Post-Implantation Mouse Development at the Single-Cell Level" published October 2018 in Cell: https://doi.org/10.1016/j.cell.2018.09.031. The images can be viewed online in the [Image Data Resource](http://idr.openmicroscopy.org/webclient/?show=project-502).

Some images from the Image Data Resource (IDR) have been converted into Zarr using [bioformats2raw](https://github.com/glencoesoftware/bioformats2raw) and stored on a public S3 store.

What is **Zarr**? [Zarr](https://zarr.readthedocs.io/en/stable/) is a young format for the storage of chunked, compressed, N-dimensional arrays.

We will use [Dask](https://docs.dask.org/en/latest/), a flexible library for parallel computing in Python. 

### Install dependencies if required
The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). 

In [ ]:
%pip install omero-py dask_image zarr fsspec>=0.3.3 aiohttp

### Import 

In [1]:
import dask
import dask.array as da
import dask_image.ndfilters
import dask_image.ndmeasure
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
%matplotlib inline

### ID of the IDR image to analyze
The original image ID is used to identify the file on the S3 store.

In [4]:
image_id = 4007801

### Load Zarr Image from S3

The method will return a dask array without any binary data. The dimension order is ``(TCZYX)``. Data will be loaded when requested during the analysis.

In [2]:
def load_binary_from_s3(id, resolution='4'):
    endpoint_url = 'https://uk1s3.embassy.ebi.ac.uk/'
    root = 'idr/zarr/v0.1/%s.zarr/%s/' % (id, resolution)
    return da.from_zarr(endpoint_url + root)

In [5]:
# Load the dask array
data = load_binary_from_s3(image_id)

In [7]:
# Check the dimension of the array
print(data.shape)

(532, 2, 988, 128, 135)


### Segment the image
This could be replaced by the analysis you wish to perform on a plane.

In [8]:
def analyze(t, c, z):
    plane = data[t, c, z, :, :] 
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
    name = "t:%s, c: %s, z:%s" % (t, c, z)
    return label_image, name

### Use Dask Delayed 

We use ``dask.delayed`` on our function so it records what we want to compute as a task into a graph that will run later on parallel hardware.

Due to the size of the image, we only analyze in the context of this notebook a few planes around the middle z-section and middle timepoint for the first channel. In total, we are analyzing 100 planes.

In [9]:
lazy_results = []
middle_z = data.shape[2] // 2
middle_t = data.shape[0] // 2
range_t = 5
range_z = 5
range_c = 1
for t in range(middle_t - range_t, middle_t + range_t):
    for z in range(middle_z - range_z, middle_z + range_z):
        for c in range(range_c):
            lazy_result = dask.delayed(analyze)(t, c, z)
            lazy_results.append(lazy_result)
print(lazy_results)

[Delayed('analyze-3a88bfbf-a462-4ece-8864-fe08e218a769'), Delayed('analyze-fdfb0923-c9b2-46bb-a047-de00766157c1'), Delayed('analyze-fad31d2e-e4d5-42f7-a296-8b33275b0eac'), Delayed('analyze-1150e10e-efea-432b-8737-a0b3f59175f3'), Delayed('analyze-94d6c7ec-ac02-44ea-b8df-dd3632f2efcf'), Delayed('analyze-b8cd0a5b-b194-4b29-9a6d-55f57f10d2d7'), Delayed('analyze-600c6ff3-4357-49f7-a5f8-7847f60c0cc9'), Delayed('analyze-e46abfd1-4a7b-45e8-9234-63170e4f3c87'), Delayed('analyze-25f4e1ff-56f8-4d60-a447-2b38ea48cbd4'), Delayed('analyze-78b9c9a7-90f0-421d-ba12-b4ed6323658b'), Delayed('analyze-49e09a50-e214-416b-8d80-787ec41cb8ee'), Delayed('analyze-908c97da-9529-4f96-9274-09dd345164a8'), Delayed('analyze-6dcdbde7-f7e2-41dd-b086-b2fdd530a188'), Delayed('analyze-8c4c4fba-1dec-4331-8797-a6c979243b8f'), Delayed('analyze-0b7d2c19-e855-4cea-acd0-05322463f6cf'), Delayed('analyze-dd7e3627-7187-4561-b291-e3783400f147'), Delayed('analyze-a0a72496-d606-481f-921d-ff6fbf089402'), Delayed('analyze-50b0892b-b308

### Run in parallel

The lazy_results list contains information about ``number_t*number_z*number_c`` calls to our function ``analyze`` that have not yet been run. We call ``dask.compute`` when we want the results.
The binary data is loaded from the S3 store during the ``compute`` phase and the analysis is performed.

In [10]:
%time results = dask.compute(*lazy_results)

CPU times: user 1min 42s, sys: 4.93 s, total: 1min 47s
Wall time: 1min 48s


### Display the results

Display the segmented planes. Use the slider to select the plane.

In [14]:
from ipywidgets import *

def display(i=0):
    r, name = results[i]
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(r)
    plt.title(name)
    fig.canvas.flush_events()

interact(display, i= widgets.IntSlider(value=0, min=0, max=len(results)-1, step=1, description="Select Plane", continuous_update=False))


interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Plane', max=99), Output(…

<function __main__.display(i=0)>

### License (BSD 2-Clause)
Copyright (C) 2020-2022 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.